In [1]:
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown
from langchain import PromptTemplate
from langchain.llms import AzureOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)

In [2]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             openai_api_version=OPENAI_DEPLOYMENT_VERSION,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):

    llm = AzureOpenAI(deployment_name=deployment_name,
                      model=model,
                      openai_api_version=openai_api_version,
                      base_url=OPENAI_DEPLOYMENT_ENDPOINT,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      top_p=top_p,
                      model_kwargs={"stop": ["<|im_end|>"]}
                      )
    return llm
llm = init_llm()

c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\llms\openai.py:853: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://aidemos-workshop.openai.azure.com/ to https://aidemos-workshop.openai.azure.com/openai.
  warnings.warn(
c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\llms\openai.py:860: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\llms\openai.py:868: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of

### **Retain conversation history** 

##### The Large Language Models (LLMs) are stateless. This means that they don’t retain any information about the conversation history.
Each transaction is independent of the previous one. Chatbots keep in memory the conversation history and use it to generate the next response. This is why they are able to generate more coherent responses.

Previously we saw that a model fails to answer the question that requires context. We can solve this problem by retaining the conversation history. We can do this by using the LangChain ConversationBufferMemory.

In [3]:
template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:

            <|im_end|>
            """

In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate

# ConversationBufferMemory is a memory that stores the conversation history
memory = ConversationBufferMemory()
# try to change the verbose to True, to see more details
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

In [5]:
prompt = ChatPromptTemplate.from_template(template=template)

response = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                                                expertise="Risk Management",
                                                question="How do you use technology or specific financial tools to assist in risk management for your clients?"))

display(Markdown(response))

 Hello! I'm happy to help. There are many ways that technology and financial tools can be used to assist in risk management. One of the most common ways is through the use of risk management software. This software can help identify potential risks and provide tools to mitigate those risks. Additionally, there are many financial tools that can be used to help manage risk, such as options, futures, and other derivatives. These tools can be used to hedge against potential losses and protect against market volatility. Finally, there are many data analysis tools that can be used to help identify trends and patterns in financial data, which can be used to make more informed decisions about risk management. Does that help?

In [6]:
# Now with the conversation history, the model can answer questions that require context.
response = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                                                expertise="Risk Management",
                            question="Which software do you use? List software products in a separate line."))

display(Markdown(response))

  There are many different risk management software products available, and the specific software that we use depends on the needs of our clients. Some popular options include:

- RiskMetrics
- MSCI RiskManager
- Algorithmics RiskWatch
- SAS Risk Management for Banking
- Moody's RiskCalc

Each of these software products has its own strengths and weaknesses, and the best choice will depend on the specific needs of the client. Does that help?

In [7]:
# you can print the conversation history buffer:
history = conversation.memory.chat_memory.messages
for msg in history:
    print(f"{msg.type}: {msg.content}")

human: Human: You are a Financial Trading Consultant answering users questions. 
            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:

            <|im_end|>
            
ai:  Hello! I'm happy to help. There are many ways that technology and financial tools can be used to assist in risk management. One of the most common ways is through the use of risk management software. This software can help identify potential risks and provide tools to mitigate those risks. Additionally, there are many financial tools that can be used to help manage risk, such as options, futures, and other derivatives. Th

In [8]:
# pay attention that LangChain added to the prompt: ```Current conversation:{history}```
print(conversation.prompt)
print()
print(memory.load_memory_variables({}))

input_variables=['history', 'input'] template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

{'history': "Human: Human: You are a Financial Trading Consultant answering users questions. \n            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.\n            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.\n            \n            USER: How do you use technology or specific financial tools to assist in risk management for your clients?\n            ASSISTANT:\n\n            <|im_end|>\n            \nAI:  Hello! I'm happy to help. There are many ways that technology and 

In [9]:
ans = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                                           expertise="Risk Management",
                                           question="List all questions I've asked you about Risk Management?"))
display(Markdown(ans))

  Sure, here are the questions you've asked me so far:

- How do you use technology or specific financial tools to assist in risk management for your clients?
- Which software do you use? List software products in a separate line.
- List all questions I've asked you about Risk Management?

Is there anything else I can help you with?

#### More conversation memory types.

Keeping full conversation history could be expensive and we can hit the Azure API limits. LangChain provides different types of conversation memory that can be used to keep the conversation history and mitigate the limits issues.

**1. ConversationBufferWindowMemory** - keeps the last N messages in the conversation history.

**2. ConversationTokenBufferMemory** - keeps the last N tokens in the conversation history.

**3. ConversationSummaryBufferMemory** - keeps summary of the conversation over time.

Additional Memory Type includes:

**4. Vector data memory** - stires the text in a vector DB and retrieves most semantically similar text.

**5. Entity memories** - use LLM which rememebers details about specific entities. 

Note: You can use multiple memories at the same time. 

Finally you can store the conevrsation history in a conventional database like SQL or NoSQL. 


In [10]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "What's your name"},
                    {"output": "My name is John"})

memory.load_memory_variables({})

{'history': "Human: What's your name\nAI: My name is John"}